## Import Packages

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms, utils
from torch.autograd import Variable
import umap
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import json
import glob
import os
from learner import Learner

# from advertorch.attacks import LinfPGDAttack
%matplotlib inline

device = torch.device("cuda:0")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0, 0, 0,), std=(1, 1, 1))
])

In [ ]:
# s = (224-2)//2
# s = (s-2)//2
# s = s-3
# config = [
#     ('conv2d', [32, 3, 3, 3, 1, 0]),
#     ('relu', [True]),
#     ('bn', [32]),
#     ('max_pool2d', [2, 2, 0]),
#     ('conv2d', [32, 32, 3, 3, 1, 0]),
#     ('relu', [True]),
#     ('bn', [32]),
#     ('max_pool2d', [2, 2, 0]),
#     ('conv2d', [32, 32, 3, 3, 1, 0]),
#     ('relu', [True]),
#     ('bn', [32]),
#     ('max_pool2d', [2, 1, 0]),
#     ('flatten', []),
#     ('linear', [5, 32 * s * s])
# ]
# # vars = torch.load(args.pretrained).to(device)

# model = Learner(config, 3, 224)
# model = model.to(device)
# model.load_state_dict(torch.load("../model/conv3_5way_224.pt"))

# print(model)

## Load Model

In [ ]:
model = torch.load("../model/resnet18_5way_112.pt")
model = model.to(device)

print(model)

## Load Data

In [ ]:
PATH = "eps_6.0"
image_paths = glob.glob(f"../{PATH}/*", )
image_names = [os.path.basename(i) for i in image_paths]

## Split Model && Inference

In [ ]:
errorses = []
vectors = []
labels = []

with torch.no_grad():

    for k in range(len(image_paths)//2):
        x = Image.open(image_paths[2 * k]) # attacked
        x = transform(x).to(device).unsqueeze(0)

        x2 = Image.open(image_paths[2 * k + 1]) # original
        x2 = transform(x2).to(device).unsqueeze(0)

        model_children = list(model.children())
        counter = 0
        names = []
        outputs = []
        errors = []
        # vectors = []
                
        for i in range(len(model_children)-1):
            if type(model_children[i]) == nn.Sequential:
                for j in range(len(model_children[i])):
                    x = model_children[i][j](x)
                    x2 = model_children[i][j](x2)
                    for child in model_children[i][j].children():
                        #if type(child) == nn.Conv2d:
                        counter+=1
                        # model_weights.append(child.weight)
                        names.append(str(child)[:str(child).index("(")] + str(len(list(filter(lambda x:str(child)[:str(child).index("(")] in x, names)))))
                        # print(str(child)[:str(child).index("(")] + str(len(list(filter(lambda x:str(child)[:str(child).index("(")] in x, names)))))
                        outputs.append(x)
                        outputs.append(x2)
                        outputs.append(torch.abs(x-x2))
                        errors.append(torch.abs(x-x2))

            else:
                x = model_children[i](x)
                x2 = model_children[i](x2)
                # if type(model_children[i]) == nn.Conv2d:
                counter+=1
                # model_weights.append(model_children[i].weight)
                names.append(str(model_children[i])[:str(model_children[i]).index("(")] + str(len(list(filter(lambda x:str(model_children[i])[:str(model_children[i]).index("(")] in x, names)))))
                outputs.append(x)
                outputs.append(x2)
                outputs.append(torch.abs(x-x2))
                errors.append(torch.abs(x-x2))

                if i == len(model_children)-2: # feature
                    vectors.append(x.view(-1))
                    vectors.append(x2.view(-1))
                    labels.append(int(image_names[2 * k].split("_")[-1][:-4]))
                    labels.append(-1*int(image_names[2 * k + 1].split("_")[-1][:-4]))

        errorses.append(errors[:])
        # print(erros[0][0][0][0][0])

        # print(counter)


## Make Feature Map

In [ ]:
LAYER = 10
FEATURE_MAP_NUM = 64

processed = []
for feature_map in outputs[LAYER * 3:(LAYER + 1) * 3]:
    feature_map = feature_map.squeeze(0)
    gray_scale = torch.sum(feature_map,0)
    gray_scale = gray_scale / feature_map.shape[0]
    for features in feature_map[:FEATURE_MAP_NUM]:
        processed.append(features.data.cpu().numpy())
    # processed.append(gray_scale.data.cpu().numpy())
# fig = plt.figure(figsize=(11, 500))

fig = plt.figure(figsize=(200, 11))

# fig = plt.figure(figsize=(50, 50))
# for i in range(len(processed)):

print(len(processed))
for i in range(len(processed)):
    a = fig.add_subplot(3, FEATURE_MAP_NUM, i+1)
    imgplot = plt.imshow(processed[i], vmin=0, vmax=1)
    a.axis("off")
    # a.set_title(f"{names[i//3]}_{i//3}", fontsize=10)
plt.savefig(f'feature_maps_{PATH}_conv_{LAYER}.jpg', bbox_inches='tight')

## Make Error Graph

In [ ]:
print(len(errorses))
mean_errorses = []

for errors in errorses:
    mean_errors = []
    for error in errors:
        _, c, h, w = error.shape

        error_sum = torch.sum(error, (1, 2, 3))
        error_mean = torch.abs((error_sum / (c * h * w))).item()
        
        mean_errors.append(error_mean)
        # print(error.shape)
        # print(error_mean)

    mean_errorses.append(mean_errors)

    # print()



In [ ]:
mean_errorses_tensor = torch.tensor(mean_errorses)

mean_errorses_mean = torch.mean(mean_errorses_tensor, dim=0)

print(mean_errorses_mean.shape)

In [ ]:
plt.figure(figsize=(20, 6))  # Width: 10 inches, Height: 6 inches

# Create the plot
plt.plot(names, mean_errorses_mean)
plt.xticks(fontsize=5)

# plt.ylim(0, 1.0)

plt.savefig('error.png')

## Make UMAP Visualization

In [ ]:
features = torch.stack(vectors).cpu()

print(features.shape)

In [ ]:
reducer = umap.UMAP(n_components=2, n_neighbors=8, min_dist=0.3, metric="euclidean", random_state=50)

embedding = reducer.fit_transform(features)
print(embedding.shape)

In [ ]:
colors = ['red', 'blue', 'green', 'orange', 'purple']
markers = ['o', 's', '^', 'x', 'D']

for i in range(embedding.shape[0]//2):
    # print(labels[i])

    plt.scatter(embedding[2 * i,0], embedding[2 * i,1], label=abs(labels[2 * i]), marker=markers[labels[2 * i + 1]], color="blue", s=40) # attack
    plt.scatter(embedding[2 * i + 1,0], embedding[2 * i + 1,1], label=abs(labels[2 * i + 1]), marker=markers[labels[2 * i + 1]], color="orange", s=40) # origin
    
plt.savefig('diff_6.0.png')
